In [3]:
import pandas as pd

# Cargar el archivo
df = pd.read_csv('empleo_formal.csv')

# Mostrar las primeras filas
print(df.head())

# Info general del dataset
print("\n🔍 Información general:")
print(df.info())

# Descripción estadística de columnas numéricas
print("\n📊 Estadísticas descriptivas:")
print(df.describe())

# Ver valores únicos por columna categórica
print("\n🧩 Valores únicos por columna:")
for col in df.select_dtypes(include='object').columns:
    print(f"{col}: {df[col].nunique()} valores únicos")


   ID  Departamento  Latitud  Longitud Categoría     Valor
0   1     Antioquia   6.2442  -75.5812         C  0.515449
1   2     Atlántico  10.9685  -74.7813         A  0.970181
2   3  Cundinamarca   4.6814  -74.1371         C  0.160697
3   4        Nariño   1.2136  -77.2811         A  0.250591
4   5  Cundinamarca   4.6814  -74.1371         B  0.462103

🔍 Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            200 non-null    int64  
 1   Departamento  200 non-null    object 
 2   Latitud       200 non-null    float64
 3   Longitud      200 non-null    float64
 4   Categoría     200 non-null    object 
 5   Valor         200 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 9.5+ KB
None

📊 Estadísticas descriptivas:
               ID     Latitud    Longitud       Valor
count  200.000000  200.

In [17]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Cargar datos
df = pd.read_csv("empleo_formal.csv")

# App
app = Dash(__name__)
app.title = "Empleo Formal Colombia"

app.layout = html.Div([
    html.H1("Dashboard Empleo Formal", style={"textAlign": "center", "color": "white"}),

    html.Div([
        html.Label("Selecciona tipo de gráfico:", style={"color": "white"}),
        dcc.Dropdown(
            id="tipo_grafico",
            options=[
                {"label": "Barras por Categoría", "value": "barras"},
                {"label": "Boxplot por Categoría", "value": "box"},
                {"label": "Mapa Colombia", "value": "mapa"}
            ],
            value="barras"
        )
    ], style={"width": "50%", "margin": "auto"}),

    dcc.Graph(id="grafico")
], style={"backgroundColor": "#141627", "padding": "20px", "fontFamily": "Arial"})

@app.callback(
    Output("grafico", "figure"),
    Input("tipo_grafico", "value")
)
def actualizar_grafico(tipo):
    if tipo == "barras":
        # Promedio del Valor por Departamento
        df_promedio = df.groupby("Departamento")["Valor"].mean().reset_index()
        fig = px.bar(df_promedio, x="Departamento", y="Valor", 
                     title="Promedio del Valor por Departamento", 
                     color="Valor", color_continuous_scale="viridis")
        
    elif tipo == "box":
        # Distribución por Categoría
        fig = px.box(df, x="Categoría", y="Valor", 
                     title="Distribución de Valor por Categoría", 
                     color="Categoría")
        
    else:
        df_promedio = df.groupby("Departamento").agg({
            "Valor": "mean",
            "Latitud": "mean",
            "Longitud": "mean"
        }).reset_index()

        fig = px.scatter_mapbox(
            df_promedio,
            lat="Latitud",
            lon="Longitud",
            color="Valor",
            size="Valor",
            hover_name="Departamento",
            zoom=4.5,
            center={"lat": 4.5709, "lon": -74.2973},
            mapbox_style="carto-darkmatter",
            title="Promedio del Valor por Departamento"
        )

    fig.update_layout(paper_bgcolor="#141627", plot_bgcolor="#141627", font_color="white")
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)